In [85]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot


sb.set() # set the default Seaborn style for graphics

In [86]:
print('hellow world')

hellow world


In [87]:
energyData = pd.read_csv('owid-energy-data.csv')
energyData = pd.DataFrame(energyData)








In [88]:

import json
import ipywidgets as widgets
from ipyleaflet import Map, GeoJSON, Choropleth, basemaps, LayerGroup
import branca.colormap as cm



# read the geojson data
with open('custom.geo.json') as f:
    geo_json_data = json.load(f)
    for d in geo_json_data["features"]:
        d["name"] = d["properties"]["name_sort"]

# get the min and max year available
min_year = energyData['year'].min()
max_year = energyData['year'].max()
print(min_year)
print(max_year)




1900
2022


In [89]:
# create the base map
m = Map(center=(0, 0), zoom=2, basemap=basemaps.CartoDB.Positron)

# create a slider for the year
year_slider = widgets.IntSlider(min=min_year, max=max_year, step=1, value=min_year)

# create a colormap
color_scale = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=0, vmax=energyData['gdp'].max())

def update_choropleth(year):
    # filter the data for the current year and remove the rows where the GDP value is missing
    choro_data = energyData.loc[(energyData['year'] == year) & (energyData['gdp'].notna()),]
    choro_data  = dict(zip(choro_data["country"].str.strip(), choro_data["gdp"]))
    # create a choropleth layer
    choropleth_layer = Choropleth(
        geo_data=geo_json_data,
        choro_data=choro_data,
        colormap=color_scale,
        style={'fillOpacity': 0.6, 'weight': 0.5},
        name='GDP Choropleth',
        key_on="name"
    )
    
    # remove the previous choropleth layer (if any) and add the new one
    layer_group.clear_layers()
    layer_group.add_layer(choropleth_layer)

# create a layer group to hold the choropleth layer
layer_group = LayerGroup()
m.add_layer(layer_group)

# add the slider to the map
widgets.jslink((year_slider, 'value'), (m, 'zoom')) # to avoid slider to steal the focus
widgets.interact(update_choropleth, year=year_slider)

# display the map
m




interactive(children=(IntSlider(value=1900, description='year', max=2022, min=1900), Output()), _dom_classes=(…

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…